In [1]:
import numpy as np
import pandas as pd
import random
import torch
import os
import argparse
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, get_linear_schedule_with_warmup

import lightning.pytorch as pl
import torchmetrics

from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger

2023-10-12 01:38:21.878933: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-12 01:38:21.934573: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-12 01:38:22.695403: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class NewsDataset(Dataset):
    def __init__(self, dataset):
        self.labels = dataset['label']
        self.input_ids = dataset['input_ids']
        self.attention_mask = dataset['attention_mask']
        self.totken_type_ids = dataset['token_type_ids']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        input_ids = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        token_type_ids = self.totken_type_ids[idx]
        return {"label": label, 
                "input_ids": input_ids, 
                "attention_mask": attention_mask,
                "token_type_ids": token_type_ids,
               }

In [3]:
def load_data(trainset_file_path, testset_file_path):
    train_data = pd.read_csv(trainset_file_path, encoding='utf-8')
    test_data = pd.read_csv(testset_file_path, encoding='utf-8')
    
    if train_data.isnull().sum().sum() != 0:
        train_data = train_data.dropna()
        train_data = train_data.reset_index(drop=True)

    if test_data.isnull().sum().sum() != 0:
        test_data = test_data.dropna()
        test_data = test_data.reset_index(drop=True)
    
    return train_data, test_data

In [4]:
def preprocessing_data(dataset, tokenizer, max_length):
    headline = [sent for sent in dataset['Headline']]
    content = [sent for sent in dataset['Content']]
        
    tokenized_sentences = tokenizer(
        headline, content,
        return_tensors = "pt",
        padding = True,
        truncation = True,
        max_length = max_length,
        add_special_tokens = True,
        return_token_type_ids= True,
    )

    input_ids = tokenized_sentences.input_ids
    attention_mask = tokenized_sentences.attention_mask
    token_type_ids = tokenized_sentences.token_type_ids
    label = torch.tensor(dataset['Class'])
    
    return {
        "label": label,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "token_type_ids": token_type_ids,
    }

In [5]:
def get_dataloader(train_path, test_path, tokenizer, MAX_LEN, BATCH_SIZE):
    # 데이터 파일 로드
    train_data, test_data = load_data(train_path, test_path)
    
    # 토크나이징 & 데이터셋 로드
    tokenized_train = NewsDataset(preprocessing_data(train_data, tokenizer, MAX_LEN))
    tokenized_test = NewsDataset(preprocessing_data(test_data, tokenizer, MAX_LEN))
    
    # train, valid split
    generator = torch.Generator().manual_seed(RANDOM_SEED)
    train, valid = random_split(tokenized_train, [0.8, 0.2], generator=generator)
    
    # DataLoader
    train_dataloader = DataLoader(train, sampler=RandomSampler(train), batch_size=BATCH_SIZE, num_workers=8)
    valid_dataloader = DataLoader(valid, sampler=SequentialSampler(valid), batch_size=BATCH_SIZE, num_workers=8)
    test_dataloader = DataLoader(tokenized_test, sampler=SequentialSampler(tokenized_test), batch_size=BATCH_SIZE, num_workers=8)
    
    return train_dataloader, valid_dataloader, test_dataloader

In [6]:
class Model(pl.LightningModule):

    def __init__(self, MODEL_NAME, total_steps, train_only_fc=False):
        super(Model, self).__init__()
        model_config = AutoConfig.from_pretrained(MODEL_NAME)
        model_config.num_labels = 2
        model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=model_config,
                                                                  ignore_mismatched_sizes=True)

        self.total_steps = total_steps
        
        if train_only_fc:
            for param in model.parameters():
                param.requires_grad = False
            for param in model.classifier.parameters():
                param.requires_grad = True

        self.model = model
            
        self.accuracy = torchmetrics.classification.BinaryAccuracy()
        self.precision = torchmetrics.classification.BinaryPrecision()
        

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
                self.model.parameters(),
                lr=2e-5,
                betas=(0.9, 0.999),
                eps=1e-08,
                )

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps = 0,
            num_training_steps = self.total_steps)
        
        return dict(
            optimizer = optimizer,
            lr_scheduler = dict(scheduler=scheduler, interval='step')
        )

    def _shared_step(self, batch, batch_idx):
        labels = batch["label"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]

        output = self.model(
                input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                labels=labels
                )
        return {'loss':output['loss'], 
                'labels':labels, 
                'logits': output['logits']
                }
        
    def training_step(self, batch, batch_idx):
        output = self._shared_step(batch, batch_idx)

        train_loss = output['loss']
        train_acc = self.accuracy(torch.argmax(output['logits'], dim=1), output['labels'])
        train_precision = self.precision(torch.argmax(output['logits'], dim=1), output['labels'])

        self.log('train_loss', train_loss)
        self.log('train_acc_step', train_acc)
        self.log('train_precision_step', train_precision)

        return output


    def validation_step(self, batch, batch_idx):
        output = self._shared_step(batch, batch_idx)

        valid_loss = output['loss']
        valid_acc = self.accuracy(torch.argmax(output['logits'], dim=1), output['labels'])
        valid_precision = self.precision(torch.argmax(output['logits'], dim=1), output['labels'])

        self.log('valid_loss', valid_loss)
        self.log('valid_acc_step', valid_acc)
        self.log('valid_precision_step', valid_precision)

        return output

    def test_step(self, batch, batch_idx):
        output = self._shared_step(batch, batch_idx)
        
        test_acc = self.accuracy(torch.argmax(output['logits'], dim=1), output['labels'])
        test_precision = self.precision(torch.argmax(output['logits'], dim=1), output['labels'])

        self.log('test_acc_step', test_acc)
        self.log('test_precision_step', test_precision)

        return output

In [7]:
def seed_everything(seed):
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

In [14]:
MODEL_NAME = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
TRAIN_PATH = "~/Data/data_whole.csv"
TEST_PATH = "~/Data/VL_data.csv"
MAX_LEN = 256
BATCH_SIZE = 16
TRAIN_FC_ONLY = False

In [15]:
seed_everything(42)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
RANDOM_SEED = 42

In [11]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_dataloader, valid_dataloader, test_dataloader = get_dataloader(TRAIN_PATH, TEST_PATH, tokenizer, MAX_LEN, BATCH_SIZE)
total_steps = len(train_dataloader) * 20

model = Model(MODEL_NAME, total_steps, train_only_fc=TRAIN_FC_ONLY)
ckpt_dir = 'model/ckpt'
ckpt_filename = MODEL_NAME + '-{epoch}-{valid_loss}'
    
wandb_logger = WandbLogger(project="mahimahi")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at Huffon/klue-roberta-base-nli and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Currently logged in as: 8chatea8 (mahimahi). Use `wandb login --relogin` to force relogin


In [12]:
early_stop_callback = EarlyStopping(
        monitor='valid_loss',
        patience=5,
        verbose=True,
        mode='min'
    )

model_checkpoint = ModelCheckpoint(
        dirpath=ckpt_dir,
        filename=ckpt_filename,
        save_top_k = 1,
        monitor='valid_loss',
        mode='min'
    )
    
callbacks = [early_stop_callback, model_checkpoint]
    
trainer = pl.Trainer(logger=wandb_logger,
                     check_val_every_n_epoch=1,
                     callbacks = callbacks,
                     max_epochs=20)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(model, train_dataloader, valid_dataloader)

You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/aiffelmahimahi/anaconda3/envs/mahimahi/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /home/aiffelmahimahi/model/ckpt exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                             | Params
---------------------------------------------------------------
0 | model     | RobertaForSequenceClassification | 110 M 
1 | accuracy  | BinaryAccuracy                   | 0     
2 | precision | BinaryPrecision                  | 0     
----

Sanity Checking: 0it [00:00, ?it/s]

../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [96,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [96,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [96,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [96,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [96,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [96,0,0], thread: [5,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1146: indexSelectLargeIndex: block: [96,0,0], thread:

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [21]:
trainer.test(ckpt_path="best", dataloaders=test_dataloader)

Restoring states from the checkpoint path at /home/aiffelmahimahi/model/ckpt/bert-base-multilingual-cased-epoch=2-valid_loss=0.29712021350860596.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/aiffelmahimahi/model/ckpt/bert-base-multilingual-cased-epoch=2-valid_loss=0.29712021350860596.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_acc_step       │     0.663302481174469     │
│    test_precision_step    │    0.7323967218399048     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc_step': 0.663302481174469,
  'test_precision_step': 0.7323967218399048}]